# Stable Diffusion - Konzepte

Für die Demonstration werden wir `Automatic1111` verwenden.
Wir werden dabei nicht auf alle Features eingehen können.
Detaillierte Informationen sind im [Projekt-Wiki](https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Features) zu finden oder in diesem [Showcase](https://github.com/AUTOMATIC1111/stable-diffusion-webui-feature-showcase).

Wer einige der Modelle und Features von Stable Diffusion online ausprobieren möchte, kann einen Account of [NightCafe](https://creator.nightcafe.studio/) erstellen und kostenlos ein paar Bilder generieren.

## Vom Text zum Bild: Txt2Img

<div style="text-align: center; margin-top: 1rem;">
    <img src="images/istanbul.jpeg" width=300 />
    <div>
        <a href="https://civitai.com/images/3224460"> nexus99@Civitai.com </a>
    </div>
</div>


### Prompts

* zenraler Teil der (Text- und) Bildgenerierung
* neuer Beruf: Prompt Engineer
* Promptkomposition
  + spezifischer Inhalt
  + Medium (Foto, Gemälde)
  + Stil (ggfs. nach Künstler)
  + Anmerkungen und Details
  + Gewichtung nutzen

```
Chaotic stunning istanbul city, skyline, illustrated by hergé, style of tin tin comics, pen and ink. vintage 90's anime style, colorful clouds, symmetric
```

### Gewichtung (Attention)

* ein `(Wort)` - erhöht die Gewichtung auf 1.1
* ein `((Wort))` - erhöht die Gewichtung erneut um den Faktor 1.1 auf 1.21 (= 1.1 * 1.1)
* ein `[Wort]` - reduziert die Gewichtung um den Faktor 1.1 
* ein `(Wort:1.5)` - erhöht die Gewichtung um den Faktor 1.5
* ein `(Wort:0.25)` - reduziert die Gewichtung um den Faktor 4 (= 1 / 0.25)
* um runde Klammern im Prompt zu benutzen, müssen diese 'escaped' werden: `\(Wort\)`

### Prompt Chunks and BREAK

Ein Prompt ist normalerweise auf 75 Tokens begrenzt.
Wird ein längerer Prompt verwendet, wird ein neues Segment/ein neuer "Chunk" erzeugt und der Prompt in mehrer Teile zerlegt.
Jeder Chunk wird unabhängig von anderen Chunks vorverarbeitet und danach mit den Ergebnissen der anderen Chungs kombiniert, bevor die Eingabe dem Modell übergeben wird.
Um Inhalte zu gruppieren oder semantisch zu trennen, kann ein Chunk mit dem Codewort `BREAK` auch vor Erreichen der 75-Token-Grenze beendet werden.

### Negative Prompt

* fast genauso wichtig wie der normale Prompt
* großer Einfluss auf den Inhalt und die Ästhetik
* weniger rigide gestaltet

```
(worst quality, low quality, normal quality, lowres, low details, oversaturated, undersaturated, overexposed, underexposed, grayscale, bw, bad photo, bad photography, bad art:1.4), (watermark, signature, text font, username, error, logo, words, letters, digits, autograph, trademark, name:1.2), (blur, blurry, grainy), morbid, ugly, asymmetrical, mutated malformed, mutilated, poorly lit, bad shadow, draft, cropped, out of frame, cut off, censored, jpeg artifacts, out of focus, glitch, duplicate, (bad hands, bad anatomy, bad body, bad face, bad teeth, bad arms, bad legs, deformities:1.3)
```

### Manche Konzepte sind schwer zu vermitteln #Challenge

<div style="text-align: center; margin-top: 1rem;">
    <img src="images/racoons.jpg" width=300 />
</div>


```
a photo of three raccoons in a trench coat pretending to be one human, high resolution, city, street, sunny weather
```


### Stable Diffusion ist eine **K**ompos**I**tion 

* CLIP -> Text zu Zahlen
* Base -> Generierung des initialen Bildes aus visuellem niedrigaufgelösten Rauschen
* Refiner -> Herausarbeitung von Bilddetails auf hochaufgelöstem Bild (optional)
* VAE -> Autoencoder; Kodierungs- und Dekodierungshilfe zur Verbesserung der Bildqualität (optional)


### Grundlegende Parameter 

* **CFG Scale**: bestimmt wie genau sich der Bildgenerierungsprozess an den Prompts orientieren soll. "Näher am Prompt" bedeutet aber nicht, dass das Bild ästhetisch ansprechender ist. Werte größer als `15` werden teils sehr surreal. Kleinere Werte wie bspw. `2` erzeugen 'natürlichere' Bilder, die den Trainingsdaten näher kommen.
* **Bildformat** [w/h]: die Größe des anfänglichen Rauschens, unterschiedliche Seitenverhältnisse verändern die Ausgaben teils drastisch
  * hängen vom gewählten Model ab (SD 1.5 512x512; SDXL 1024x1024)
  * große Bilder = mehr Speicherbedarf = längere Berechnungen
* **Sampling Steps**: die Anzahl der Iteration, eine hohe Anzahl erzeugt mehr Details, bedeutet aber auch eine längere Bearbeitungszeit
* **Batch Size**: Wie viele Bilder sollen pro Durchlauf berechnet werden
* **Batch Count**: Wie viele Durchläufe sollen durchgeführt werden
* **Refiner**: gibt an, wann soll das Basismodel durch das *Refiner*-Modell ersetzt werden
* **VAE**: muss erst über die Settings aktiviert werden


### Weitere Parameter

* **Sampling method**: die genutzte Methode, um aus dem zufällig erzeugten Bild das Rauschen schrittweise zu entfernen
  * Euler A (Standard), DPM++ 2M Karras (schnell), DDIM (schneller), DPM2 a (langsam)
  * bei vielen Samlplingschritten konvergieren viele Sampler zu einem ähnlichen Bild
* **Clip Skip**: Wenn höher als 1 wird die Konvertierung des Textprompts früher abgebrochen
* **Eta Noise Seed Delta (ENSD)**: Zufallsfaktor bei der Generierung verschiedener Bilder, relevant für Reproduzierbarkeit
* **Upscaling/Hires.fix**: Entfernt Artefakte aus dem Bild und erhöht die Auflösung
  * **Face Restoration**: Herunter- und wieder Hochskalierung mit GFPGAN or CodeFormer von Bildteilen in denen Gesichter vermutet werden

### Prompt 'Tweaks'

#### Temporäre Schlüsselwörter

* `[to:when]` - adds to to the prompt after a fixed number of steps (when)
* `[from::when]` - removes from from the prompt after a fixed number of steps (when)

`a [fantasy:cyberpunk:16] landscape`

#### Prompt matrix

Promptsegmente mit `|` unterteilt werden verschiedene Prompts erzeugt. Das erlaubt das schnelle Ausprobieren verschiedener Kombinationen:

`a busy city street in a modern city|illustration|cinematic lighting prompt`

* a busy city street in a modern city
* a busy city street in a modern city, illustration
* a busy city street in a modern city, cinematic lighting
* a busy city street in a modern city, illustration, cinematic lighting

## Vom Bild (und Text) zum Bild: Img2Img

### Grundlegende Parameter

* **Denoising Strength**: die Mengen and Bildrauschen, die hinzugefügt wird, um das Bild zu verändern. `0` fügt kein Rauschen hinzu, sodass Eingabe und Ausgabe (meistens) identisch ist. Ein Wert von `1` ersetzt das Eingabebild komplett.
* **Resize mode**: gibt an, wie das Bild skaliert werden soll, geeignet um Bildausschnitte herauszuarbeiten
  - **Just resize**: ignoriert das Seitenverhältnis
  - **Crop and resize**: schneidet das Bild erst zu und skaliert dann
  - **Resize and fill**: fügt Bildrauschen hinzu und skaliert dann

### Outpainting

Methode um Randbereiche zu einem Bild hinzuzufügen.

#### Beispielvorgehen

* Bild und Prompt nach `img2img` übertragen
* Sampling method: `Euler a`
* Resize Mode: `Resize and fill`
* Script: `Outpainting mk2`
* Sampling Steps: `80`
* CFG Scale: `30`
* Denoising Strength: `0.8`
* Nach Bedarf Outpaint Direction einstellen

### Inpainting

Überzeichnet Bereiche eines bestehenden Bildes.

#### Beispielvorgehen

* Bild und Prompt nach `inpaint` kopieren
* Bereich maskieren
* Mask Blur: `4` (SD 1.5) bzw. `8` (SDXL)
* Inpaint area: `Only masked`
* Prompt mit Details des maskierten Bereichs erweitern